<a href="https://colab.research.google.com/github/thad75/OptionAI/blob/main/TP%202%20Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2 : Sentiment Analysis

In [ ]:
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# we fix the seeds to get consistent results before every training
# loop in what follows
def fix_seed(seed=234):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)

## About this lab

In this notebook we are going to build state-of-the art models for text classification using the example of sentiment analysis. To be more precise, we will build a feed-forward neural network (FFNN) and a convolutional neural network (CNN). We will look into the details of data preparation, functioning of each model and how the performance of those NNs could be measured efficiently. We will start our work using a toy corpus. Further you can extend your knowledge and use a larger dataset.

Again we are using [pytorch](https://www.pytorch.org), an open source deep learning platform, as our backbone library in the course.

Goal of this lab : 
* Understand word representation
* Practice your Deep Learning Skills
* Train a model for NLP


## Toying around

We present to you our toy datasets : the toy training and validation sets. It is good practice to use the validation set (a representative set of the test data). This set is used to tune hyperparameters and choose a configuration for your model to ensure the best performance. 

Our toy sets are already tokenized and lowercased.

In [ ]:
 # Our toy sentiment analysis corpus
train = ['i like his new book !',
         'what a well-written novel !',
         'i do not agree with the criticism on this short story',
         'well done ! it was an enjoyable stage play',
         'it was very good . send me a copy please .',
         'the argumentation in the study is very weak',
         'poor effort !',
         'the descriptions could have been more detailed',
         'i am not impressed',
         'could have done better .',
]

train_labels = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

# Validation set
valid = ['i like your play', 
         'i agree with your study', 
         'what a success ! a well-written novel', 
         'not enough details . very poor', 
         'i support the criticism',
         'could be better',
]

valid_labels = [1, 1, 1, 0, 0, 0]

### Pre-processing

* Using simple list operators, fill in the below function '...' to tokenize the corpus:

In [ ]:
def get_tokenized_corpus(corpus):
  tokenized_corpus = []

  #######################
  # Q: Process the corpus
  #######################
  for sentence in corpus:
    tokenized_sentence = 
    for token in sentence.split(' '): 
      tokenized_sentence....(token)
    tokenized_corpus....(tokenized_sentence)
 
  return tokenized_corpus

### Word2index dictionary

In order to easily retrieve each word in the vocabulary, we define here a method that returns a word to index dictionary. Note that we reserve the 0 index for the padding token `<pad>`.

In [ ]:
def get_word2idx(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
  
  word2idx = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  # we reserve the 0 index for the padding token
  word2idx['<pad>'] = 0
  
 
  return word2idx

### Preparation of inputs

The first layer of our FFNN will be an embedding (look-up) layer which takes as input indexes of tokens (we do not need to one-hot encode our vectors).

---

**Q: Why do we need to fix the length of our input vectors (we take the maximum sentence length here) ? This process is referred to as padding. Print the padded training corpus.**

In [ ]:
def get_model_inputs(tokenized_corpus, word2idx, labels):
  # we index our sentences
  vectorized_sents = [[word2idx[tok] for tok in sent if tok in word2idx] for sent in tokenized_corpus]

  # Sentence lengths
  sent_lengths = [len(sent) for sent in vectorized_sents]

  # Get maximum length
  max_len = max(...)
  
  # we create a tensor of a fixed size filled with zeroes for padding
  sent_tensor = torch.zeros((len(vectorized_sents), max_len)).long()

  # we fill it with our vectorized sentences 
  for idx, (sent, sentlen) in enumerate(zip(vectorized_sents, sent_lengths)):
    sent_tensor[idx, :sentlen] = torch.LongTensor(sent)

  # Label tensor
  label_tensor = torch.FloatTensor(...)
  
  return sent_tensor, label_tensor

###

tokenized_corpus = get_tokenized_corpus(train)
word2idx = get_word2idx(...)
train_sent_tensor, train_label_tensor = get_model_inputs(tokenized_corpus, word2idx, train_labels)

## Model 1 : the Feed-Forward Neural Network

We will start by building a very simple feed-forward neural network (FFNN).
Our FFNN class is a sub-class of `nn.Module`. Within the `__init__` method, we define the layers of the module:

- Our first layer is an embedding layer (look-up layer). This layer could be initialized with pre-trained embeddings (as we will see at the end of this lab) or could be trained together with other layers.
 
- The next layer is a fully connected layer followed by a ReLU activation.

- Finally, the last linear layer is the output layer for the classification task.

The `forward()` method is called when we feed data into our model. Please note that the output dimension of each layer is the input dimension for the next one.

---


**Q: Implement the averaging of embeddings in the `forward()` method of the class below.**


In [ ]:
class FFNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, num_classes):  
        super(FFNN, self).__init__()
        
        # embedding (lookup layer) layer
        # padding_idx argument makes sure that the 0-th token in the vocabulary
        # is used for padding purposes i.e. its embedding will be a 0-vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # hidden layer
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        
        # activation
        self.relu1 = nn.ReLU()
        
        # output layer
        self.fc2 = nn.Linear(hidden_dim, num_classes)  
    
    def forward(self, x):
        # x has shape (batch_size, max_sent_len)

        embedded = self.embedding(x)
        # `embedding` has shape (batch size, max_sent_len, embedding dim)

        ########################################################################
        # Q: Compute the average embeddings of shape (batch_size, embedding_dim)
        ########################################################################
        # Implement averaging that ignores padding (average using actual sentence lengths).
        # How this effect the result?
        
        sent_lens = ...
        averaged = ... / sent_lens

        out = self.fc1(averaged)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

### Training the model

In this section we will define the hyperparameters of our model, the loss function, the optimizer and perform a number of training epochs over our toy training corpus.

We will use the **Stochastic gradient descent (SGD)** optimizer. The learning rate hyperparameter of the optimizer controls how the weights are adjusted with respect to the loss gradient. The lower the value, the more fine-grained are weight updates.

**Note that** it is a common practise to perform training using mini-batches (sets of training instances seen by the model during weight update step). In this case, the epoch loss is defined as the loss averaged across the mini-batches. Since our corpus is very small, we train on the whole training set without batching.

---

**Q: Why is the number of output classes is equal to 1 for binary classification?**


**Q: Try to modify the learning rate (which is initially set to 0.5 below) in the range $[0.0001, 0.5]$. How does the loss react to these changes?**

In [ ]:
# Reset the seed before every model construction for reproducible results
fix_seed()

# we will train for N epochs (The model will see the corpus N times)
EPOCHS = 10

# Learning rate is initially set to 0.5
LRATE = 0.5

# we define our embedding dimension (dimensionality of the output of the first layer)
EMBEDDING_DIM = 50

# dimensionality of the output of the second hidden layer
HIDDEN_DIM = 50

# the output dimension is the number of classes, 1 for binary classification
OUTPUT_DIM = 1

# Construct the model
model = FFNN(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), OUTPUT_DIM)

# Print the model
print(model)

# we use the stochastic gradient descent (SGD) optimizer
optimizer = optim.SGD(model.parameters(), lr=LRATE)

# we use the binary cross-entropy loss with sigmoid (applied to logits) 
# Recall that we did not apply any activation to our output layer, hence we need
# to make our outputs look like probabilities.
loss_fn = nn.BCEWithLogitsLoss()

# Input and label tensors
feature = train_sent_tensor
target = train_label_tensor

################
# Start training
################
print(f'Will train for {EPOCHS} epochs')
for epoch in range(1, EPOCHS + 1):
  # to ensure the dropout (explained later) is "turned on" while training
  # good practice to include even if do not use here
  model.train()
  
  # we zero the gradients as they are not removed automatically
  optimizer.zero_grad()
  
  # squeeze is needed as the predictions will have the shape (batch size, 1)
  # and we need to remove the dimension of size 1
  predictions = model(feature).squeeze(1)

  # Compute the loss
  loss = loss_fn(predictions, target)
  train_loss = loss.item()

  # calculate the gradient of each parameter
  loss.backward()

  # update the parameters using the gradients and optimizer algorithm 
  optimizer.step()
  
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f}')

### Measuring the accuracy

In addition to measuring the loss, we can also evaluate the actual classification performance of our model. (In the case of training with mini-batches, the epoch accuracy is defined as the accuracy averaged across the mini-batches.)

---

**Q: Fill in the below function so that it computes the accuracy of the model. Once you are done, improve the loop so that it also prints the training accuracy after each epoch.**

In [ ]:
def accuracy(output, target):
  #####################################
  # Q: Return the accuracy of the model
  #####################################
  acc = ...
  return acc

In [ ]:
# Reset the seed for consistent results
fix_seed()

# we will train for N epochs (The model will see the corpus N times)
EPOCHS = 10

# Learning rate is initially set to 0.5
LRATE = 0.5

# we define our embedding dimension (dimensionality of the output of the first layer)
EMBEDDING_DIM = 50

# dimensionality of the output of the second hidden layer
HIDDEN_DIM = 50

# the output dimension is the number of classes, 1 for binary classification
OUTPUT_DIM = 1

# Construct the model
model = FFNN(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), OUTPUT_DIM)

# Print the model
print(model)

# we use the stochastic gradient descent (SGD) optimizer
optimizer = optim.SGD(model.parameters(), lr=LRATE)

# we use the binary cross-entropy loss with sigmoid (applied to logits) 
# Recall that we did not apply any activation to our output layer, hence we need
# to make our outputs look like probabilities.
loss_fn = nn.BCEWithLogitsLoss()

# Input and label tensors
feature = train_sent_tensor
target = train_label_tensor

################
# Start training
################
print(f'Will train for {EPOCHS} epochs')
for epoch in range(1, EPOCHS + 1):
  # to ensure the dropout (explained later) is "turned on" while training
  # good practice to include even if do not use here
  model.train()
  
  # we zero the gradients as they are not removed automatically
  optimizer.zero_grad()
  
  # squeeze is needed as the predictions will have the shape (batch size, 1)
  # and we need to remove the dimension of size 1
  predictions = model(feature).squeeze(1)

  # Compute the loss
  loss = loss_fn(predictions, target)
  train_loss = loss.item()
  
  #####################
  # Q: Compute accuracy
  #####################
  train_acc = ...

  # calculate the gradient of each parameter
  loss.backward()

  # update the parameters using the gradients and optimizer algorithm 
  optimizer.step()

  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

### Hyperparameter tuning on the validation set

You should now apply the previous pre-processing and input preparation procedures to the validation set as well.

---

**Q: Should we re-use the word to index dictionary we created before? Why?**
 

In [ ]:
###############################################
# Q: Prepare the validation corpus and labels #
###############################################
print(valid_sent_tensor)

**Q: Try to modify the learning rate and the number of epochs now. How will the validation loss and accuracy react to those changes?**

In [ ]:
# Reset the seed for consistent results
fix_seed()

# we will train for N epochs (The model will see the corpus N times)
EPOCHS = 10

# Learning rate is initially set to 0.5
LRATE = 0.5

# we define our embedding dimension (dimensionality of the output of the first layer)
EMBEDDING_DIM = 50

# dimensionality of the output of the second hidden layer
HIDDEN_DIM = 50

# the output dimension is the number of classes, 1 for binary classification
OUTPUT_DIM = 1

# Construct the model
model = FFNN(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), OUTPUT_DIM)

# we use the stochastic gradient descent (SGD) optimizer
optimizer = optim.SGD(model.parameters(), lr=LRATE)

# we use the binary cross-entropy loss with sigmoid (applied to logits) 
# Recall that we did not apply any activation to our output layer, hence we need
# to make our outputs look like probabilities.
loss_fn = nn.BCEWithLogitsLoss()

# Input and label tensors for training
feature_train = train_sent_tensor
target_train = train_label_tensor

# Input and label tensors for validation
feature_valid = valid_sent_tensor
target_valid = valid_label_tensor

################
# Start training
################
print(f'Will train for {EPOCHS} epochs')
for epoch in range(1, EPOCHS + 1):
  # to ensure the dropout (explained later) is "turned on" while training
  # good practice to include even if do not use here
  model.train()
  
  # we zero the gradients as they are not removed automatically
  optimizer.zero_grad()
  
  # squeeze is needed as the predictions will have the shape (batch size, 1)
  # and we need to remove the dimension of size 1
  predictions = model(feature_train).squeeze(1)

  # Compute the loss
  loss = loss_fn(predictions, target_train)
  train_loss = loss.item()

  # Compute training accuracy
  train_acc = accuracy(predictions, target_train)

  # calculate the gradient of each parameter
  loss.backward()

  # update the parameters using the gradients and optimizer algorithm 
  optimizer.step()
  
  # this puts the model in "evaluation mode" (turns off dropout and batch normalization)
  # good practise to include even if we do not use them right now
  model.eval()

  # we do not compute gradients within this block, i.e. no training
  with torch.no_grad():
    predictions_valid = model(feature_valid).squeeze(1)
    valid_loss = loss_fn(predictions_valid, target_valid).item()
    valid_acc = accuracy(predictions_valid, target_valid)
  
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:6.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:6.2f}% |')

### Testing the model

Now let us test our trained model. We define a small test set below. First, apply the data preparation procedures to this test set as you did for the validation set.


In [ ]:
test = ['i really do not like your short story', 
        'well done', 
        'good results for a study !',
        'amazing effort', 
        'your effort is poor !', 
        'not impressed'   
]

test_labels = [0, 1, 1, 1, 0, 0]

#########################################
# Q: Prepare the test corpus and labels #

print(test_sent_tensor)

**Q: Fill in the below function for the computation of F-measure. Once done, complete the missing lines in the final evaluation part.**

In [ ]:
def f_measure(output, gold):
  ############################################
  # Q: Compute precision, recall and f-measure 
  ############################################
  pred = torch.round(torch.sigmoid(output))
  pred = pred.detach().cpu().numpy()
     
  test_pos_preds = np.sum(pred)
  test_pos_real = np.sum(gold)
    
  correct = (np.logical_...pred, gold)).astype(int)
  correct = np.sum(correct)
  
  precision = correct / test_pos_...
  recall = correct / test_pos_...
  
  fscore = (2.0 * precision * recall) / (... + ...)

  # Print them
  print(f"     Recall: {recall:.2f}, Precision: {precision:.2f}, F-measure: {fscore:.2f}")
  

####

model.eval()

feature_test = test_sent_tensor
target_test = test_label_tensor

with torch.no_grad():
  ####################################################################
  # Q: Get predictions for the test set, compute the loss and accuracy
  ####################################################################
  predictions = ...
  test_loss = ...
  test_acc = ...

  # Print
  print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
  f_measure(predictions, test_labels)  Recall

**Q:  Are the resulting evaluations different ? How do you interpret those differences? Print the predictions.**
 

## Model 2 : the Convolutional Neural Network (CNN)

We will implement a model inspired by the CNN model as described in [Convolutional Neural Networks for Sentence Classification (Kim, 2014)](https://arxiv.org/abs/1408.5882).

Similar to the FFNN model, we start with an embedding layer. We implement the convolutional layer with the help of `nn.Conv2d` and use the ReLU activation after it. The above-mentioned paper, being inspired by the convolution for images, applies a 2-dimensional convolution: a (window size, embedding dimension) filter. It covers `n` sequential words, taking embedding dimensions as the width. We then pass the tensors through a **max pooling layer**.

The **max pooling layer** is typically followed by a **dropout** layer. The latter sets a random set of activations in the max-pooling layer to zero. This prevents the network from learning to rely on specific weights and helps to prevent overfitting. Note that the dropout layer is only used during training, and not during test time.

---

**Q: Study the shapes of outputs coming from convolution and max pooling layers. What is the shape of the max pooling layer output?**

In [ ]:
class CNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim, out_channels, window_size, output_dim, dropout):
    super(CNN, self).__init__()
    
    # Create the embedding layer as usual
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
    
    # in_channels -- 1 text channel
    # out_channels -- the number of output channels
    # kernel_size is (window size x embedding dim)
    self.conv = nn.Conv2d(
      in_channels=1, out_channels=out_channels,
      kernel_size=(window_size, embedding_dim))
    
    # the dropout layer
    self.dropout = nn.Dropout(dropout)

    # the output layer
    self.fc = nn.Linear(out_channels, output_dim)
        
  def forward(self, x):
    # x -> (batch size, max_sent_length)
    
    embedded = self.embedding(x)
    # embedded -> (batch size, max_sent_length, embedding_dim)
    
    # images have 3 RGB channels 
    # for the text we add 1 channel
    embedded = embedded.unsqueeze(1)
    # embedded -> (batch size, 1, max_sent_length, embedding dim)

    # Compute the feature maps      
    feature_maps = self.conv(embedded)

    ##########################################
    # Q: What is the shape of `feature_maps` ?
    ##########################################
    
    feature_maps = feature_maps.squeeze(3)
    
    ##########################################
    # Q: why do we remove 1 dimension here?
    ##########################################
    
    # Apply ReLU
    feature_maps = F.relu(feature_maps)
    
    # Apply the max pooling layer
    pooled = F.max_pool1d(feature_maps, feature_maps.shape[2])
    
    pooled = pooled.squeeze(2)

    ####################################
    # Q: What is the shape of `pooled` ?
    ####################################
    
    dropped = self.dropout(pooled)
    preds = self.fc(dropped)
    
    return preds

### Training and testing the CNN

Here we will define the CNN-specific hyperparameters and perform the network training and testing. **Note that** the learning rate is initially set to 0.1.

In [ ]:
fix_seed()

EPOCHS = 10
LRATE = 0.1

EMBEDDING_DIM = 50
OUTPUT_DIM = 1

# the hyperparameters specific to CNN
# we define the number of filters
N_OUT_CHANNELS = 100

# we define the window size
WINDOW_SIZE = 1

# we apply the dropout with the probability 0.2
DROPOUT = 0.2

# Construct the model
model = CNN(len(word2idx), EMBEDDING_DIM, N_OUT_CHANNELS, WINDOW_SIZE, OUTPUT_DIM, DROPOUT)

optimizer = optim.SGD(model.parameters(), lr=LRATE)
loss_fn = nn.BCEWithLogitsLoss()

feature_train = train_sent_tensor
target_train = train_label_tensor

feature_valid = valid_sent_tensor
target_valid = valid_label_tensor

feature_test = test_sent_tensor
target_test = test_label_tensor

################
# Start training
################
print(f'Will train for {EPOCHS} epochs')
for epoch in range(1, EPOCHS + 1):
  model.train()
  
  optimizer.zero_grad()

  predictions = model(feature_train).squeeze(1)
  loss = loss_fn(predictions, target_train)
  train_loss = loss.item()
  train_acc = accuracy(predictions, target_train)

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    predictions_valid = model(feature_valid).squeeze(1)
    valid_loss = loss_fn(predictions_valid, target_valid).item()
    valid_acc = accuracy(predictions_valid, target_valid)
  
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:6.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:6.2f}% |')


## Finally, test on the test set
model.eval()

with torch.no_grad():
    predictions = model(feature_test).squeeze(1)
    loss = loss_fn(predictions, target_test)
    acc = accuracy(predictions, target_test)
    print(f'Test Loss: {loss:.3f} | Test Acc: {acc*100:.2f}%')
    f_measure(predictions, test_labels)

 **Q: Is the performance of CNN different from the performance of FFNN? Output predictions.**
 
**Q: Is padding necessary for CNN inputs? What is the role of the window size?**

### Initializing CNN with pre-trained representations

The work [Convolutional Neural Networks for Sentence Classification (Kim, 2014)](https://arxiv.org/abs/1408.5882) also investigates the exploitation of pre-trained embeddings and demonstrates the efficiency of using them.

First, download the embeddings and unzip them below:

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
# Unzip the file: 4 different embedding sizes are provided
!unzip glove.6B.zip

In [ ]:
# Check the file format
!head -n10 glove.6B.50d.txt


Try and initialize the CNN embedding layer with the `50D` pre-trained GloVe embeddings. Pay particular attention to keeping the correct indices from the `word2idx` for the lookup table! Once you fill the below `wvecs` matrix, copy the previous training loop and initialize its embedding layer with the pre-trained ones as follows:

```python
model.embedding.weight.data.copy_(torch.from_numpy(wvecs))
```

**Note:** The learning rate is initially set to 0.5.

---

**Q: What should the embedding for the padding token `<pad>` be?**
 
**Q: What is the impact of using those pre-trained embeddings on the model performance?**

In [ ]:
from tqdm import tqdm

EMBEDDING_DIM = 50

# Yet another hyperparameter: since the pre-trained embeddings are coming
# from a different network, their magnitudes could differ from the parameters
# of this network. So scaling may be necessary.
SCALE_EMBS = 0.65

# Creates the empty numpy array that you should fill below
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')

#####################################################################
# Q: Read line by line, find the corresponding word and
# insert its embedding to the correct position in the `wvecs` matrix.
# Once done, apply the SCALE_EMBS factor to scale the vectors
#####################################################################
with open(f'glove.6B.{EMBEDDING_DIM}d.txt', 'r') as f:
  "<TODO>"
        
print(wvecs)

#####################
# Re-create the model
#####################
fix_seed()

EPOCHS = 10
LRATE = 0.5

# the hyperparameters specific to CNN
OUTPUT_DIM = 1

# we define the number of filters
N_OUT_CHANNELS = 100

# we define the window size
WINDOW_SIZE = 1

# we apply the dropout with the probability 0.1
DROPOUT = 0.1

# Construct the model
model = CNN(len(word2idx), EMBEDDING_DIM, N_OUT_CHANNELS, WINDOW_SIZE, OUTPUT_DIM, DROPOUT)

#################################################################
### Q: Initialize the embeddings with the loaded pre-trained ones
#################################################################
...

optimizer = optim.SGD(model.parameters(), lr=LRATE)
loss_fn = nn.BCEWithLogitsLoss()

feature_train = train_sent_tensor
target_train = train_label_tensor

feature_valid = valid_sent_tensor
target_valid = valid_label_tensor

feature_test = test_sent_tensor
target_test = test_label_tensor

################
# Start training
################
print(f'Will train for {EPOCHS} epochs')
for epoch in range(1, EPOCHS + 1):
  model.train()
  
  optimizer.zero_grad()

  predictions = model(feature_train).squeeze(1)
  loss = loss_fn(predictions, target_train)
  train_loss = loss.item()
  train_acc = accuracy(predictions, target_train)

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    predictions_valid = model(feature_valid).squeeze(1)
    valid_loss = loss_fn(predictions_valid, target_valid).item()
    valid_acc = accuracy(predictions_valid, target_valid)
  
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:6.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:6.2f}% |')


## Finally, test on the test set
model.eval()

with torch.no_grad():
    predictions = model(feature_test).squeeze(1)
    loss = loss_fn(predictions, target_test)
    acc = accuracy(predictions, target_test)
    print(f'Test Loss: {loss:.3f} | Test Acc: {acc*100:.2f}%')
    f_measure(predictions, test_labels)